In [1]:
import osmnx as ox
import pandas as pd
import numpy as np

In [2]:
# longitude is x, latitude is y

G = ox.graph_from_place('San Francisco, CA, USA', network_type='drive')
nodes = ox.graph_to_gdfs(G, edges=False)

In [3]:
temp_cols = list(nodes.columns)
temp_cols[temp_cols.index('x')] = 'lng'; temp_cols[temp_cols.index('y')] = 'lat'
nodes.columns = temp_cols
nodes = nodes.sort_index()

In [4]:
# create mapping from original node number names to new node number names

node_name_map = {}; node_name_map_inv = {}
node_og = list(nodes.index)
num_nodes = len(node_og)
for i in range(num_nodes):
    node_name_map[i] = node_og[i]
    node_name_map_inv[node_og[i]] = i
nodes.index = range(nodes.shape[0])

In [5]:
df_police_stops = pd.read_csv('ca_san_francisco_2019_08_13.csv')
df_police_stops = df_police_stops[pd.notnull(df_police_stops['lng'])]
df_police_stops = df_police_stops[pd.notnull(df_police_stops['lat'])]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
def haversine_np(long1, lat1, long2, lat2):
    """
    Calculate the great circle distance between an array of points to a single point
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    long1, lat1, long2, lat2 = map(np.radians, [long1, lat1, long2, lat2])

    dlon = long2 - long1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [7]:
from scipy.spatial import distance as dist
from time import time

coords = nodes[['lng', 'lat']].values
long1 = np.array(coords)[:, 0]
lat1 = np.array(coords)[:, 1]
batch_size = 500
police_stops = {}
m = df_police_stops.shape[0]
max_police_stops = 0
for i in range(0, m, batch_size):
    if i % 10000 == 0: print('%d/%d' % (i, m))
    if i + batch_size >= m:
        stop_coords = df_police_stops[['lng', 'lat']].values[i:]
    else:
        stop_coords = df_police_stops[['lng', 'lat']].values[i:i+batch_size]
    
    associated_nodes = np.array([haversine_np(long1, lat1, long2, lat2) for (long2, lat2) in stop_coords]).argmin(axis=1)
#     associated_nodes = dist.cdist(coords, stop_coords).argmin(axis=0)
    for idx in range(len(associated_nodes)):
        if associated_nodes[idx] in police_stops:
            new_police_stop = np.vstack([police_stops[associated_nodes[idx]], df_police_stops.iloc[i+idx][['subject_sex', 'subject_race']].values])
        else:
            new_police_stop = np.reshape(df_police_stops.iloc[i+idx][['subject_sex', 'subject_race']].values, [1, 2])
        
        if new_police_stop.shape[0] > max_police_stops: max_police_stops = new_police_stop.shape[0]
        police_stops[associated_nodes[idx]] = new_police_stop
#        police_stops[associated_nodes[idx]] = np.vstack([police_stops.get(associated_nodes[idx], ), df_police_stops.iloc[i+idx][['subject_sex', 'subject_race']].values])#police_stops.get(associated_nodes[idx], []) + [df_police_stops.iloc[i+idx][['subject_sex', 'subject_race']].values]
        

0/903373
10000/903373
20000/903373
30000/903373
40000/903373
50000/903373
60000/903373
70000/903373
80000/903373
90000/903373
100000/903373
110000/903373
120000/903373
130000/903373
140000/903373
150000/903373
160000/903373
170000/903373
180000/903373
190000/903373
200000/903373
210000/903373
220000/903373
230000/903373
240000/903373
250000/903373
260000/903373
270000/903373
280000/903373
290000/903373
300000/903373
310000/903373
320000/903373
330000/903373
340000/903373
350000/903373
360000/903373
370000/903373
380000/903373
390000/903373
400000/903373
410000/903373
420000/903373
430000/903373
440000/903373
450000/903373
460000/903373
470000/903373
480000/903373
490000/903373
500000/903373
510000/903373
520000/903373
530000/903373
540000/903373
550000/903373
560000/903373
570000/903373
580000/903373
590000/903373
600000/903373
610000/903373
620000/903373
630000/903373
640000/903373
650000/903373
660000/903373
670000/903373
680000/903373
690000/903373
700000/903373
710000/903373
720000

In [8]:
max_edges = 6
rewards = {}
worse_reward = -1000
for node in range(num_nodes):
    edges = list(G[nodes.iloc[node]['osmid']]) # G has the edges as the original node numbers thats what osmid is
#     max_edges = max(max_edges, len(edges))
    # convert the edges from original names to new renamed, sorted is just to keep actions in some kind of order
    # for example the 1st action will always be the first chronological node based on naming
    edges = sorted([node_name_map_inv[edge] for edge in edges]) 
    # set reward of going to this edge equal to negative number of police stops at this node
    # if there are no police stops at the edge set reward to 0
    rewards[node] = [-(police_stops[edge].shape[0])/max_police_stops if edge in police_stops else 0 for edge in edges]
    rewards[node] = rewards[node] + (max_edges-len(rewards[node]))*[worse_reward]
    
# fill terrible reward for nodes that have less total actions, encodes impossible actions essentially
# for node in range(num_nodes):
#     curr_actions = rewards[node]
#     rewards[node] = curr_actions + [worse_reward]*(max_edges-len(curr_actions))

In [9]:
# np.unique(df_police_stops['subject_race'].values)
from threading import Thread


num_features = 2
def calc_reward(edge, sex, race):
#     if len(edge.shape) == 1: edge = edge.reshape(1, num_features)
    matching_sex = np.sum(edge[:, 0]==sex)
    matching_race = np.sum(edge[:, 1]==race)
    return -((.5 + .5*(matching_sex + matching_race)/num_features)*edge.shape[0])/max_police_stops

sexes = np.unique(df_police_stops['subject_sex'])
races = np.unique(df_police_stops['subject_race'].values)

def fill_state(node):
    edges = list(G[nodes.iloc[node]['osmid']]) # G has the edges as the original node numbers thats what osmid is
    edges = sorted([node_name_map_inv[edge] for edge in edges])
    for sex in sexes:
        for race in races:
            state = (node, sex, race)
            police_rewards_plus[state] = [calc_reward(police_stops[edge], sex, race) if edge in police_stops else 0 for edge in edges]
            police_rewards_plus[state] = police_rewards_plus[state] + (max_edges-len(police_rewards_plus))*[worse_reward]
    
    print("Finished thread for node %d" % node)


police_rewards_plus = {}
nodes_list = list(range(num_nodes))

def check_threads(threads):
    count = 0
    for i in reversed(range(len(threads))):
        if not threads[i].isAlive():
            del threads[i]
            count += 1
    
    return count

threads = []
max_threads = 100

print("Starting %d threads" % max_threads)
for t in range(max_threads):
    next_node = nodes_list.pop(0)
    threads.append(Thread(target=fill_state, args=(next_node,)))

    threads[-1].start()
    
print("Finished starting %d threads" % max_threads)

while(len(nodes_list) > 0):
    count = check_threads(threads)
    if count > 0:
        for c in range(count):
            if len(nodes_list) == 0: break
            next_node = nodes_list.pop(0)
            
            threads.append(Thread(target=fill_state, args=(next_node,)))
            print("Starting thread for node %d" % next_node)
            threads[-1].start()
    else:
        continue
        
for t in threads:
    t.join()
# for node in range(num_nodes):
    
    

Starting 100 threads
Finished thread for node 0
Finished thread for node 1
Finished thread for node 2
Finished thread for node 3
Finished thread for node 4
Finished thread for node 5
Finished thread for node 6
Finished thread for node 7
Finished thread for node 8
Finished thread for node 9
Finished thread for node 10
Finished thread for node 11
Finished thread for node 12
Finished thread for node 13
Finished thread for node 14
Finished thread for node 15
Finished thread for node 16
Finished thread for node 17
Finished thread for node 18
Finished thread for node 19
Finished thread for node 20
Finished thread for node 21
Finished thread for node 22
Finished thread for node 23
Finished thread for node 24Finished thread for node 25

Finished thread for node 26
Finished thread for node 27
Finished thread for node 28
Finished thread for node 29
Finished thread for node 30
Finished thread for node 31
Finished thread for node 32
Finished thread for node 33
Finished thread for node 34
Finished 

Finished thread for node 294
Starting thread for node 295
Finished thread for node 295
Starting thread for node 296
Finished thread for node 296
Starting thread for node 297
Finished thread for node 297Starting thread for node 298
Finished thread for node 298
Starting thread for node 299

Finished thread for node 299
Starting thread for node 300
Finished thread for node 300
Starting thread for node 301
Finished thread for node 301
Starting thread for node 302
Finished thread for node 302Starting thread for node 303

Finished thread for node 303Starting thread for node 304
Finished thread for node 304

Starting thread for node 305
Finished thread for node 305
Starting thread for node 306
Finished thread for node 306
Starting thread for node 307
Finished thread for node 307
Starting thread for node 308
Finished thread for node 308Starting thread for node 309

Finished thread for node 309
Starting thread for node 310
Finished thread for node 310
Starting thread for node 311
Finished threa

Starting thread for node 510
Finished thread for node 510Starting thread for node 511
Finished thread for node 511
Starting thread for node 512

Finished thread for node 512
Starting thread for node 513
Finished thread for node 513Starting thread for node 514

Finished thread for node 514
Starting thread for node 515
Finished thread for node 515
Starting thread for node 516
Finished thread for node 516
Starting thread for node 517
Finished thread for node 517
Starting thread for node 518
Finished thread for node 518
Starting thread for node 519
Finished thread for node 519
Starting thread for node 520
Finished thread for node 520
Starting thread for node 521
Finished thread for node 521
Starting thread for node 522
Finished thread for node 522
Starting thread for node 523
Finished thread for node 523
Starting thread for node 524
Finished thread for node 524
Starting thread for node 525
Finished thread for node 525
Starting thread for node 526
Finished thread for node 526
Starting threa

Finished thread for node 729Starting thread for node 730
Finished thread for node 730Starting thread for node 731


Finished thread for node 731
Starting thread for node 732
Finished thread for node 732Starting thread for node 733

Finished thread for node 733Starting thread for node 734

Finished thread for node 734
Starting thread for node 735
Finished thread for node 735Starting thread for node 736
Finished thread for node 736
Starting thread for node 737
Finished thread for node 737
Starting thread for node 738

Finished thread for node 738
Starting thread for node 739
Finished thread for node 739
Starting thread for node 740
Finished thread for node 740
Starting thread for node 741
Finished thread for node 741
Starting thread for node 742
Finished thread for node 742
Starting thread for node 743
Finished thread for node 743Starting thread for node 744
Finished thread for node 744

Starting thread for node 745
Finished thread for node 745
Starting thread for node 746
Finished threa

Finished thread for node 946
Starting thread for node 947
Finished thread for node 947Starting thread for node 948

Finished thread for node 948
Starting thread for node 949
Finished thread for node 949
Starting thread for node 950
Finished thread for node 950
Starting thread for node 951
Finished thread for node 951Starting thread for node 952

Finished thread for node 952
Starting thread for node 953
Finished thread for node 953
Starting thread for node 954
Finished thread for node 954
Starting thread for node 955
Finished thread for node 955
Starting thread for node 956
Finished thread for node 956
Starting thread for node 957
Finished thread for node 957
Starting thread for node 958
Finished thread for node 958
Starting thread for node 959
Finished thread for node 959
Starting thread for node 960
Finished thread for node 960
Starting thread for node 961
Finished thread for node 961
Starting thread for node 962
Finished thread for node 962
Starting thread for node 963
Finished threa

...

Starting thread for node 8842
Finished thread for node 8842
Starting thread for node 8843
Finished thread for node 8843Starting thread for node 8844
Finished thread for node 8844

Starting thread for node 8845
Finished thread for node 8845
Starting thread for node 8846
Finished thread for node 8846
Starting thread for node 8847
Finished thread for node 8847Starting thread for node 8848
Finished thread for node 8848

Starting thread for node 8849
Finished thread for node 8849
Starting thread for node 8850
Finished thread for node 8850Starting thread for node 8851

Finished thread for node 8851
Starting thread for node 8852
Finished thread for node 8852
Starting thread for node 8853
Finished thread for node 8853Starting thread for node 8854

Finished thread for node 8854
Starting thread for node 8855
Finished thread for node 8855
Starting thread for node 8856
Finished thread for node 8856
Starting thread for node 8857
Finished thread for node 8857
Starting thread for node 8858
Finished t

Finished thread for node 9051
Starting thread for node 9052
Finished thread for node 9052
Starting thread for node 9053
Finished thread for node 9053
Starting thread for node 9054
Finished thread for node 9054
Starting thread for node 9055
Finished thread for node 9055
Starting thread for node 9056
Finished thread for node 9056Starting thread for node 9057

Finished thread for node 9057
Starting thread for node 9058
Finished thread for node 9058
Starting thread for node 9059
Finished thread for node 9059
Starting thread for node 9060
Finished thread for node 9060
Starting thread for node 9061
Finished thread for node 9061
Starting thread for node 9062
Finished thread for node 9062
Starting thread for node 9063
Finished thread for node 9063
Starting thread for node 9064
Finished thread for node 9064
Starting thread for node 9065
Finished thread for node 9065
Starting thread for node 9066
Finished thread for node 9066
Starting thread for node 9067
Finished thread for node 9067
Starting t

Finished thread for node 9196Starting thread for node 9197


Finished thread for node 9197Starting thread for node 9198

Finished thread for node 9198Starting thread for node 9199

Finished thread for node 9199Starting thread for node 9200
Finished thread for node 9200

Starting thread for node 9201
Finished thread for node 9201Starting thread for node 9202
Finished thread for node 9202

Starting thread for node 9203
Finished thread for node 9203Starting thread for node 9204
Finished thread for node 9204

Starting thread for node 9205
Finished thread for node 9205Starting thread for node 9206

Finished thread for node 9206Starting thread for node 9207
Finished thread for node 9207
Starting thread for node 9208

Finished thread for node 9208
Starting thread for node 9209
Finished thread for node 9209Starting thread for node 9210

Finished thread for node 9210
Starting thread for node 9211
Finished thread for node 9211Starting thread for node 9212
Finished thread for node 9212

Starting 

Starting thread for node 9346
Finished thread for node 9346Starting thread for node 9347

Finished thread for node 9347Starting thread for node 9348

Finished thread for node 9348Starting thread for node 9349

Finished thread for node 9349
Starting thread for node 9350
Finished thread for node 9350Starting thread for node 9351
Finished thread for node 9351

Starting thread for node 9352
Finished thread for node 9352
Starting thread for node 9353
Finished thread for node 9353
Starting thread for node 9354
Finished thread for node 9354Starting thread for node 9355
Finished thread for node 9355
Starting thread for node 9356

Finished thread for node 9356
Starting thread for node 9357
Finished thread for node 9357
Starting thread for node 9358
Finished thread for node 9358
Starting thread for node 9359
Finished thread for node 9359Starting thread for node 9360
Finished thread for node 9360
Starting thread for node 9361

Finished thread for node 9361Starting thread for node 9362
Finished th

Finished thread for node 9486
Starting thread for node 9487

Finished thread for node 9487Starting thread for node 9488
Finished thread for node 9488Starting thread for node 9489


Finished thread for node 9489Starting thread for node 9490
Finished thread for node 9490
Starting thread for node 9491

Finished thread for node 9491
Starting thread for node 9492
Finished thread for node 9492
Starting thread for node 9493
Finished thread for node 9493
Starting thread for node 9494
Finished thread for node 9494
Starting thread for node 9495
Finished thread for node 9495
Starting thread for node 9496
Finished thread for node 9496
Starting thread for node 9497
Finished thread for node 9497
Starting thread for node 9498
Finished thread for node 9498Starting thread for node 9499

Finished thread for node 9499Starting thread for node 9500

Finished thread for node 9500Starting thread for node 9501

Finished thread for node 9501Starting thread for node 9502
Finished thread for node 9502
Starting t

In [ ]:
# import pickle

# with open('nodes_with_police_stops.pickle', 'wb') as handle:
#     pickle.dump(police_stops, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('police_stop_rewards.pickle', 'wb') as handle:
#     pickle.dump(rewards, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# with open('nodes_with_police_stops.pickle', 'rb') as handle:
#     police_stops = pickle.load(handle)

In [7]:
traffic_df = pd.read_csv('Traffic_Calming_Point_Features_092014.csv')

In [8]:
h = traffic_df['the_geom'].values
buffer = 7
points  = np.array([(float(i[i.find('(')+1:i[buffer:].find(' ')+buffer]), float(i[i[buffer:].find(' ')+1+buffer:i.find(')')])) for i in h])
long, lat = points[:, 0], points[:, 1]
del traffic_df['the_geom']
traffic_df['lng'] = long
traffic_df['lat']  = lat
# [traffic_df['the_geom'].find('(')+1:traffic_df['the_geom'].find(' ')]

In [10]:
coords = nodes[['lng', 'lat']].values
long1 = np.array(coords)[:, 0]
lat1 = np.array(coords)[:, 1]
batch_size = 100
traffic_calmers = {}
max_traffic_calmers = 0
m = traffic_df.shape[0]
for i in range(0, m, batch_size):
    if i % 100 == 0: print('%d/%d' % (i, m))
    if i + batch_size >= m:
        stop_coords = traffic_df[['lng', 'lat']].values[i:]
    else:
        stop_coords = traffic_df[['lng', 'lat']].values[i:i+batch_size]
    
    associated_nodes = np.array([haversine_np(long1, lat1, long2, lat2) for (long2, lat2) in stop_coords]).argmin(axis=1)
#     associated_nodes = dist.cdist(coords, stop_coords).argmin(axis=0)
    for idx in range(len(associated_nodes)):
        new_traffic = traffic_calmers.get(associated_nodes[idx], 0) + 1
        max_traffic_calmers = max(new_traffic, max_traffic_calmers)
        traffic_calmers[associated_nodes[idx]] = new_traffic
        
        

0/453
100/453
200/453
300/453
400/453


In [12]:
max_edges = 6
rewards_traffic = {}
worse_reward = -1000
for node in range(num_nodes):
    edges = list(G[nodes.iloc[node]['osmid']]) # G has the edges as the original node numbers thats what osmid is
#     max_edges = max(max_edges, len(edges))
    # convert the edges from original names to new renamed, sorted is just to keep actions in some kind of order
    # for example the 1st action will always be the first chronological node based on naming
    edges = sorted([node_name_map_inv[edge] for edge in edges]) 
    print(edges)
    # set reward of going to this edge equal to negative number of police stops at this node
    # if there are no police stops at the edge set reward to 0
    rewards_traffic[node] = [(edge, -(traffic_calmers[edge])/max_traffic_calmers) if edge in traffic_calmers else 0 for edge in edges]
    rewards_traffic[node] = rewards_traffic[node] + (max_edges-len(rewards_traffic[node]))*[(-1, worse_reward)]
    
# fill terrible reward for nodes that have less total actions, encodes impossible actions essentially
# for node in range(num_nodes):
#     curr_actions = rewards[node]
#     rewards[node] = curr_actions + [worse_reward]*(max_edges-len(curr_actions))

[7842]
[7574, 9531]
[4767, 7908]
[6240, 7210]
[777]
[7, 7383, 7583]
[7, 7973, 8408]
[5, 6, 7456, 9524]
[7384, 7481]
[10, 7478, 7480]
[9, 11]
[10, 12, 8645]
[11, 153, 7482]
[14, 1366, 2031]
[1367, 2032]
[16]
[15, 813, 7957]
[816]
[19, 817, 818, 5397]
[18, 20, 819, 2341]
[19, 21, 820, 6480]
[20, 1805, 2605, 7746]
[56, 57, 7746, 9251]
[24, 804, 6331]
[23, 25, 805, 2241]
[24, 26, 806, 4111]
[25, 27, 807, 5929]
[26, 808, 1270]
[810]
[30, 5042, 5841]
[29, 5042, 5043]
[3237, 4336, 8703]
[33, 5639, 8703]
[32, 34, 5867]
[33]
[36, 5350, 5867]
[35, 1501, 9012]
[37, 38, 548, 549]
[37, 39, 1220, 1221]
[38, 5347, 6113, 6819]
[41, 2689, 2690, 6862]
[40, 42, 5348, 5830]
[41, 5349, 6503, 8889]
[44]
[43, 45, 5807]
[44, 5344, 6461]
[2175, 6935]
[5345, 6083, 6935, 7699]
[49, 5346, 6504, 7082]
[48, 50, 1033, 1034]
[49, 51, 8994, 8997]
[50, 410]
[53, 2751, 9594]
[52, 54, 3960]
[53]
[56, 2339, 5315]
[22, 55, 7745]
[22, 58]
[57, 59, 1805, 9252]
[58, 60, 821]
[59, 2606, 6175, 6990]
[5577, 5709, 7925, 7926]
[53

[59, 820, 1805, 2606]
[6776]
[822, 3274, 7063]
[825, 2037, 4654]
[824, 826, 1553, 2038]
[825, 1554, 4910]
[828, 1733, 5859, 7687]
[827, 829, 3886, 7687]
[828, 830, 1735, 3887]
[829, 1735, 5191]
[832, 1758, 4614, 9415]
[831, 3890, 4450]
[834, 1507, 1508]
[833, 835, 2379]
[834, 2378, 4620]
[682, 837, 5663]
[836, 838, 4135, 5664]
[837, 839, 2236, 4136, 5665]
[838, 840, 2235, 2236]
[839, 841, 7686]
[239, 840, 4106]
[239, 1266, 1267]
[655, 656, 8143]
[8294]
[2765, 5555, 8752, 8754]
[2757, 8634, 8747]
[848, 6494]
[849, 4509, 9191]
[848, 850, 4511]
[849, 851, 4512]
[850, 852, 3716, 4473]
[851, 853, 4513]
[852, 854, 3718, 4514]
[853, 855, 2211, 2212]
[95, 854, 856]
[855, 857, 7797]
[856, 858, 4515]
[857, 859, 3719, 4516]
[858, 860, 3720, 4286]
[859, 861, 8851]
[860, 3431, 3722]
[4520, 7329, 8841]
[864, 2067, 2068]
[863, 865, 3729]
[864, 866, 4522]
[865, 867, 1191, 1192]
[866, 868, 2533, 6647]
[867, 869]
[868, 870, 4524, 6648]
[869, 871, 3731, 4525]
[870, 872, 3732, 4526]
[871, 873, 3733, 4527]

[1555, 1557, 4778, 5106]
[1556, 1558, 2511]
[1557, 1559, 4779, 5107]
[1558, 1560, 4780]
[1559, 1561, 4781, 5108]
[1560, 1562, 4031]
[1561, 8138]
[1564, 3764]
[985, 3763, 3786]
[1566, 3762, 3787]
[1907, 1908]
[1568, 3587, 6560]
[1567, 1569, 3588, 9548]
[1568]
[1571, 3668, 8613]
[243, 1570, 3667]
[1573, 1602, 2565]
[1572, 1603, 6038]
[921, 1575, 1604, 5505]
[1574, 1576, 1605, 5680]
[1575, 1606, 7477, 8270]
[1582, 8353, 8354]
[1579, 1583, 9487]
[1578, 1580, 1583, 5959, 6624]
[1579, 1581, 2791, 2792]
[1580, 1582, 4771, 5787]
[1577, 1581, 4771]
[1579, 8249]
[1585, 3548, 6105, 9229]
[1584, 6501, 8610]
[602, 1587, 2583]
[604, 1586]
[1589, 5112, 7740]
[1588]
[1591, 1855, 6233]
[642, 1590, 1592, 4701]
[1591, 4318, 4702, 7109]
[1594, 2959, 7109]
[1593, 1595, 3224, 3935]
[1594]
[9122]
[1598, 6512, 7729, 9122]
[764, 1597, 1641, 1642]
[764, 1600, 4062, 7736]
[1599, 1601, 4063, 6037]
[1600, 1602, 8178]
[1572, 1601, 1603, 2564]
[920, 1573, 1602, 4064]
[920, 1574, 1605, 4179]
[1575, 1604, 1606, 4178]


[2319, 2321, 2624]
[2320, 3286, 3367]
[1715, 2323, 6211, 8175]
[2322, 3682, 3683, 8375]
[2325, 4238, 4704]
[2324]
[2327, 4842, 7692, 8170]
[2326, 2328]
[2327, 2329]
[6719]
[5406, 9290]
[2332, 2422]
[893, 2331, 4806, 5408]
[893, 2334, 4074]
[9222]
[2336, 9220, 9222]
[890, 2335, 4733]
[2249, 2250, 6465]
[1180, 2526]
[55, 85, 86]
[85, 2341, 8073]
[19, 2340, 5396]
[819, 5398, 6174, 6987]
[2946, 7097]
[2345, 2922, 7097]
[2344, 2346, 3232, 6321]
[2345, 2347, 5997]
[2346, 2348, 3233]
[2347, 2349, 4716]
[2348, 2350, 5888]
[2349, 2351, 3234]
[2350, 2352, 5686]
[2351, 2353, 6415]
[2352, 2354, 3168]
[525, 2353, 5228]
[525, 2356, 3678]
[2355, 2357, 5092, 5093]
[2356, 2358, 5213]
[2357, 2359, 5214]
[2358, 2360, 5215, 5231]
[2359, 2361, 5216, 6402]
[2360, 2362, 6401]
[2361, 2363, 5217]
[2362, 2364, 6036]
[2363, 2365, 4982, 4983]
[2364, 2366, 2848]
[2288, 2365, 5218]
[713, 2368, 2846]
[2367, 2369, 4555]
[2368, 2370, 3074]
[2369, 2371, 2845]
[2370, 2372, 4556]
[2371, 2844, 2866, 9195]
[2374, 4628, 768

[2841, 3071, 3073]
[3072, 8487]
[2369, 3075, 4555, 4556]
[1884, 3074, 4554, 4557]
[2569, 3077, 4553, 4558]
[3076, 3078, 4496, 4497]
[3002, 3077, 3432, 3433]
[1877, 3080, 4552, 4559]
[3079, 3081, 4551, 4560]
[2259, 3080, 4550, 4561]
[2259, 2874, 4549]
[2816, 8826]
[3085, 4735, 6429]
[3084]
[1823, 4742, 7664]
[507, 3842]
[2223, 3089, 4224]
[3088, 3090, 7694]
[1002, 1003, 3089, 7779]
[1003, 6493, 9276]
[1434, 3093, 6596]
[3092, 3094, 6527]
[2458, 3093, 4434]
[2458, 5775, 8293]
[2206, 3097, 6507]
[659, 3096, 6146]
[711, 5467, 8667]
[2754, 6374, 8667]
[7191]
[3102, 4442, 5105]
[3101, 4039]
[2317, 5593, 8817]
[4041, 7612, 8817, 9311]
[3106, 4835, 7611]
[3105, 3107, 4043, 4382]
[3106, 3108, 4044, 7807]
[3107, 3109, 4045, 5594]
[777, 3108, 3110, 4046]
[3109, 3111, 6021]
[1984, 3110, 4752, 5675]
[1984, 3113, 3223]
[770, 3112, 3114, 8161]
[3113, 4048, 4427, 6733]
[3116, 5598, 6733]
[3115, 3117, 4050]
[3116, 3118, 4051, 4608]
[3117, 3119, 4052, 5600]
[874, 3118, 3734, 8819]
[874, 3121, 4053, 8822

...

[1223, 4698, 6115, 6815]
[1224, 3782, 6114, 6350]
[1228, 3418, 5940, 6824]
[1686, 6840, 8858]
[5844]
[1336, 1337, 2153]
[3681, 8172, 8179]
[4284, 5874, 6728]
[289, 4287, 6755, 7313]
[4291, 6124, 9575]
[297, 4292, 5176, 6123]
[804, 2242, 5702]
[4794, 4795, 7132]
[3149, 5820, 6319]
[801, 4331, 6129]
[6128]
[1398, 2909, 4357, 6131]
[2805, 2910, 4358, 6130]
[6133]
[263, 4337, 5867, 6132]
[2889, 5350, 7995, 9008]
[2894, 3405, 4344, 5937]
[1699, 2902, 4353, 6137]
[2903, 4354, 6136, 6957]
[6139]
[1484, 1485, 6138, 8189]
[2879]
[4013]
[962, 2089, 2271, 6143]
[6142]
[5167, 6557]
[3388, 3390, 4922]
[3097]
[2651, 5116, 6148]
[5117, 5768, 6147, 9326]
[4942, 5120, 5766, 9265]
[3509, 5121, 7265]
[5431, 6004, 6152, 8805]
[1452, 5432, 6005, 6151]
[2930, 4476, 5113, 5772]
[2761, 6400]
[2503]
[695, 4137, 7926]
[1621, 4275, 4715, 4738]
[1062, 1394, 3458, 6485]
[1162, 1400, 6019, 6486]
[383, 8605]
[1406, 6487, 8838, 8865]
[5072, 9600]
[5095, 5213, 6164, 6469]
[6163]
[6166]
[5096, 6165, 6167, 6290]
[6166]


[2174, 6083, 6915]
[2176, 6085, 6916]
[2190, 4630, 6894]
[2187, 2488, 6918]
[2194, 4688, 6898]
[2184, 2400, 6919]
[2196, 2799, 6896]
[1413, 1414, 6922]
[2172, 2204, 6081]
[2177, 3780, 6926]
[46, 47, 6925]
[5300, 6753]
[6936, 7144]
[3743, 8005, 9059]
[3745, 5655, 6938]
[5656, 5660]
[8011, 8014]
[216, 564, 8960]
[565, 3452, 9461]
[432, 433, 6955]
[576, 5379, 6956]
[572, 2903, 6957]
[566, 6611, 6943]
[569, 3918, 6944]
[375, 6945, 8963]
[573, 1154, 6946]
[567, 1054, 6947]
[570, 4354, 6948]
[178, 577, 3741]
[574, 6546, 9474]
[568, 4201, 6951]
[575, 6018, 6954]
[571, 6137, 6952]
[1918, 8701]
[5368, 6718]
[6959, 9480]
[3517, 4672, 5481]
[117, 1691, 9052]
[5447, 5942, 6244]
[9027]
[2201, 7219]
[6965, 8228]
[249, 3934, 4395]
[251, 2252, 8188]
[3421, 3422, 9300]
[6538, 6539, 6969]
[1608, 6972, 8190]
[2157, 3305, 6971]
[1381, 8652, 9579]
[6446, 7362]
[1128, 4808]
[90, 4093]
[4816, 5976, 6976]
[3137, 6977]
[4407, 8491, 8496]
[4813, 8464, 8465]
[4073, 8456]
[5016, 5571, 7388]
[4270, 4271, 6982]
[14

[9099]
[5768, 9264, 9326]
[5175, 6649, 9273]
[1735, 2373]
[2373, 9195]
[464, 465, 1730]
[464, 1729, 4495]
[1267, 1268, 2380]
[840]
[827, 828, 1734]
[8065]
[282, 7690]
[283, 7689]
[5444, 6719, 7692]
[2326, 5444]
[4445, 6717]
[1002, 3089, 9276]
[9168]
[6024, 6025, 8523]
[4708]
[1679, 2160, 5840]
[47, 7082, 7701]
[6083, 7081, 7699]
[5345, 7083, 7702]
[6084, 7084, 7703]
[6085, 7704]
[3780, 7705]
[4639, 7086, 7706]
[5420, 7087, 7707]
[6703, 9232]
[864]
[1659, 7710, 7711]
[1660, 2498, 7709]
[1660, 5594, 7709]
[6316]
[]
[3399, 7880]
[7335]
[7335]
[6343]
[9208]
[2830, 7831]
[142, 8532, 9534]
[6778, 8530]
[4152, 7721]
[3324, 3325, 8366]
[8757]
[1018, 6418, 6419]
[6639, 6640, 6641]
[9561, 9608]
[3410, 5446]
[1597, 7821]
[228, 8779]
[7822, 9533]
[7090, 7731]
[7732, 7734, 7735]
[2587, 7735]
[7824]
[765, 1599, 7476]
[4082, 8950, 8990]
[1216, 1973, 3292]
[3384, 3673, 3674]
[1588, 4129, 5617]
[3150, 6188, 7131]
[9024]
[5445, 6720]
[1339]
[56, 2605, 7746]
[21, 22, 57]
[9329]
[3598]
[1009, 7748]
[118, 

[5955, 6517]
[2151, 2233, 4432]
[3205, 8516]
[5955, 6394, 8517, 8654]
[5028, 6782, 8204]
[8513, 8766, 8769]
[4027, 7319, 8514]
[6684, 7564]
[8520, 8697]
[746, 8518]
[2385, 8750, 8753]
[8545]
[2887]
[8525, 8527, 8642]
[3858, 8524, 8527]
[8529, 8642, 8643]
[8524, 8525, 8529]
[578, 8529, 8643]
[4012, 8526, 8527, 8528]
[142, 3331, 7721]
[1344, 1345, 1521]
[5917, 7720]
[8534]
[8533, 8661]
[3872, 3873, 7127, 7431]
[8540]
[5630, 8536]
[2412, 4461]
[2988, 4463]
[4335, 8537]
[1346, 1347, 6050]
[2989, 6194, 8407]
[2991, 8404]
[]
[7696, 8523]
[4611, 4612, 7248, 8680]
[3731, 4606, 5921, 9377]
[8357, 8568]
[6685, 8548]
[5250, 5251, 5853]
[3907, 8598]
[4312, 8597, 9009]
[8556, 9238]
[5435, 8247]
[8254, 8557]
[6680, 7626, 8557, 9238]
[8554, 8555, 8556]
[9238]
[1313, 4478, 7985]
[8007, 9098]
[8562, 9094]
[8560]
[3431]
[7329, 7332]
[4458, 8566]
[7105, 8565, 8567]
[1265, 2974, 8566]
[334, 8345, 8549]
[2099, 3360, 9528]
[8569]
[7379, 8570]
[3720, 4161, 5925, 7441]
[213, 7796]
[9333]
[8577]
[9336, 9337]
[

[7767]
[6330, 6398, 9258]
[2758, 6398]
[2037, 6104]
[8338, 9261]
[7442, 8322]
[9260, 9261]
[8338, 9260]
[3625, 7679, 9327]
[6149, 6645, 6646]
[2535, 5159, 7339, 9267]
[2536, 6644, 9266]
[5302, 6644, 6645]
[868, 6647, 6648]
[1190, 6647, 6649]
[6648, 9272]
[9239, 9271, 9273]
[4606, 5880, 7680, 9272]
[8671, 9277]
[2186, 6909]
[3091, 7239, 7694]
[2434, 8671]
[2051, 8500]
[6702, 8336, 9340]
[9283, 9403]
[9280, 9478]
[7544, 9403]
[1716, 7279]
[2010]
[4333, 5745, 9287]
[1119, 5745, 9288, 9289]
[5759, 9285, 9288]
[5759, 9286, 9287, 9289]
[1120, 8119, 9286, 9288]
[2330, 6226]
[4785]
[8300]
[2209, 2925]
[4285, 4601, 7441]
[4161, 9294]
[5874, 5926, 9295]
[284, 285, 9296]
[2648, 4602]
[9298]
[2156, 3425]
[261]
[1515, 3254]
[3275, 7392, 9420]
[3631, 9424]
[511]
[1940, 3552, 9307]
[9306]
[6063]
[6026, 9100]
[4629]
[3104, 7612]
[3362, 6103, 9556]
[2632, 2633]
[85, 86, 9315]
[9314, 9316, 9317]
[9315, 9318]
[86, 9316]
[86, 5932, 9316]
[1216, 1217, 7738]
[313, 9321]
[4667, 9320]
[6772, 8831, 9324]
[2071

In [17]:
import copy

def calculate_transition(sPrime, state, reward):
    if reward == -100000:
        return 0.0
    else:
        return 1.0/6.0 # this should not be 6, it should be the number of actual possible actions

iteration = 1
discount = 0.95
U = pd.DataFrame(np.zeros((num_nodes, 1)))
while(True):
    U_prev = U.copy()
    #algo 4.3 in sequential problems textbook
    for node in range(num_nodes):
        node_rewards = rewards[node]
#         U.iloc[node] = max(r + discount*sum(calculate_transition(sPrime, node, r)*U_prev.iloc[sPrime])
        for sPrime in range(num_nodes):
            for r in node_rewards:
                pass
    print('hi')
    difference = np.square(U.sub(U_prev)).values.sum()
    if difference < 1.0:
        break
    iteration += 1

hi
